In [4]:
%run __common.py
%load_ext autoreload
%autoreload 2

/iopsstor/scratch/cscs/stefschu/DSM500/github/modulus-a5275d8/modulus/distributed/manager.py:329: UserWarning: Distributed manager is already intialized
  warn("Distributed manager is already intialized")
Provided checkpoint directory /iopsstor/scratch/cscs/stefschu/DSM500/github/analysis/checkpoints does not exist, skipping load


Loaded validation datapipe of size 2912
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dir(dist)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_broadcast_buffers',
 '_cuda',
 '_device',
 '_distributed',
 '_find_unused_parameters',
 '_group_names',
 '_group_ranks',
 '_groups',
 '_initialization_method',
 '_is_initialized',
 '_local_rank',
 '_rank',
 '_shared_state',
 '_world_size',
 'broadcast_buffers',
 'cleanup',
 'create_group_from_node',
 'create_groups_from_config',
 'create_orthogonal_process_group',
 'create_process_subgroup',
 'cuda',
 'device',
 'distributed',
 'find_unused_parameters',
 'get_available_backend',
 'group',
 'group_name',
 'group_names',
 'group_rank',
 'group_size',
 'initialize',
 'initialize_env',
 'initialize_open_mpi',
 'initialize_s

In [2]:
import torch
import torch.multiprocessing as mp
import gc
from modulus.datapipes.climate.era5_hdf5_new import ERA5HDF5Datapipe
from torch.utils import data
import matplotlib.pyplot as plt
import numpy as np


In [3]:
experiments = dict()

In [4]:
%load_ext autoreload
%autoreload 2

import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
os.environ["HDF5_CHUNK_CACHE_SIZE"] = str(100 * 1024 * 1024)  # 100MB
os.environ["HDF5_SWMR_WRITE"] = "1"
os.environ["HDF5_SWMR_READ"] = "1"

import torch
import torch.multiprocessing as mp
import gc
from modulus.datapipes.climate.era5_hdf5_new import ERA5HDF5Datapipe
from torch.utils import data
import matplotlib.pyplot as plt

def list_cuda_tensors():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.is_cuda:
                print(type(obj), obj.size(), obj.dtype)
        except:
            pass  # Skip any errors

def print_memory_usage(tag=""):
    torch.cuda.synchronize()
    allocated = torch.cuda.memory_allocated() / 1024**2
    cached = torch.cuda.memory_reserved() / 1024**2
    print(f"[{tag}] Allocated: {allocated:.2f} MB | Cached: {cached:.2f} MB")

def tensor_memory(tensor):
    return tensor.element_size() * tensor.numel() / 1024**2  # Convert to MB

train = ERA5HDF5Datapipe(
    data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr",
    statistics_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/stats",
    static_data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/static",
    num_steps_ahead=10,
)

import torch.multiprocessing as mp
mp.set_start_method("spawn", force=True)

#mp.set_start_method("spawn", force=True)
train_loader = data.DataLoader(train, 
    batch_size=1,
    num_workers=4,
    prefetch_factor=4,
    pin_memory=True,
    shuffle=True)


collector = []
for i, (sample, stats) in enumerate(train_loader):
    #sample = sample.to("cuda:0").to(torch.bfloat16)

    collector.append(stats)
    worker_id = int(stats["worker_id"])
    year = int(stats["year"])
    time_total = float(stats["time_total"])
    time_probe = float(stats["time_probe"])

    print(f"Worker {worker_id} | Year {year} | Time total {time_total:2f} s | Time probed {time_probe:.2f} s | {(time_probe * 100 / time_total):.2f} %")
    if i == 30:
        break

time_totals = list()
time_probes = list()
for c in collector:
    time_totals.append(float(c["time_total"]))
    time_probes.append(float(c["time_probe"]))
time_totals = np.array(time_totals)
time_probes = np.array(time_probes)

experiments["stripped"] = (np.mean(time_totals), np.std(time_totals), np.mean(time_probes), np.std(time_probes))

print("-------")

for expriment, (total_mean, total_std, probe_mean, probe_std) in experiments.items():
    print(f"{expriment} | Mean: {total_mean:.2f} s | Std: {total_std:.2f} s | Mean: {probe_mean:.2f} s | Std: {probe_std:.2f} s | {(probe_mean * 100 / total_mean):.2f} %")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: 'ERA5HDF5Datapipe' object has no attribute 'statistics_dir'

In [16]:
import h5py

file_path = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1979.h5"

with h5py.File(file_path, "r") as f:
    swmr_enabled = f.swmr_mode
    libver = f.libver
    print(f"SWMR Enabled: {swmr_enabled}")
    print(f"File format version: {libver}")
    print(f["fields"].shape)
    print(f["fields"].dtype)
    print(f["fields"].chunks)
    data = np.array(f["fields"][0])
    # print data size in MB
    print(data.nbytes)



SWMR Enabled: True
File format version: ('v114', 'v114')
(1460, 21, 721, 1440)
float32
(1, 21, 721, 1440)
87212160


In [30]:
from pathlib import Path

base_path = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train"
base_path_new = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr"

for path in sorted(Path(base_path).glob("????.h5")):
    old_file_path = str(path)
    new_file_path = old_file_path.replace(base_path, base_path_new)

    print(old_file_path, new_file_path)

    optimized_chunk_size = (1, 21, 721, 1440)
    with h5py.File(old_file_path, "r") as old_f, h5py.File(new_file_path, "w", libver="latest") as new_f:

        old_data = old_f["fields"]
        new_data = new_f.create_dataset(
            "fields",
            shape=old_data.shape,
            dtype=old_data.dtype,
            chunks=optimized_chunk_size,
            track_times=False
        )
        new_data[:] = old_data[:]
        new_f.swmr_mode = True


/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1979.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1979.h5
/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1980.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1980.h5
/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1981.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1981.h5
/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1982.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1982.h5
/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1983.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1983.h5
/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1984.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1984.h5
/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/198

In [31]:
from pathlib import Path

base_path = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/test"
base_path_new = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/test_swmr"

for path in sorted(Path(base_path).glob("????.h5")):
    old_file_path = str(path)
    new_file_path = old_file_path.replace(base_path, base_path_new)

    print(old_file_path, new_file_path)

    optimized_chunk_size = (1, 21, 721, 1440)
    with h5py.File(old_file_path, "r") as old_f, h5py.File(new_file_path, "w", libver="latest") as new_f:

        old_data = old_f["fields"]
        new_data = new_f.create_dataset(
            "fields",
            shape=old_data.shape,
            dtype=old_data.dtype,
            chunks=optimized_chunk_size,
            track_times=False
        )
        new_data[:] = old_data[:]
        new_f.swmr_mode = True


/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/test/2016.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/test_swmr/2016.h5
/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/test/2017.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/test_swmr/2017.h5


In [32]:
from pathlib import Path

base_path = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/out_of_sample"
base_path_new = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/out_of_sample_swmr"

for path in sorted(Path(base_path).glob("????.h5")):
    old_file_path = str(path)
    new_file_path = old_file_path.replace(base_path, base_path_new)

    print(old_file_path, new_file_path)

    optimized_chunk_size = (1, 21, 721, 1440)
    with h5py.File(old_file_path, "r") as old_f, h5py.File(new_file_path, "w", libver="latest") as new_f:

        old_data = old_f["fields"]
        new_data = new_f.create_dataset(
            "fields",
            shape=old_data.shape,
            dtype=old_data.dtype,
            chunks=optimized_chunk_size,
            track_times=False
        )
        new_data[:] = old_data[:]
        new_f.swmr_mode = True


/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/out_of_sample/2018.h5 /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/out_of_sample_swmr/2018.h5


In [9]:
import h5py

old_file = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1979.h5"
new_file = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1979.h5"

with h5py.File(old_file, "r") as old_f, h5py.File(new_file, "w", libver="latest") as new_f:
    optimized_chunk_size = (1, 21, 721, 1440)

    old_data = old_f["fields"]
    new_data = new_f.create_dataset(
        "fields",
        shape=old_data.shape,
        dtype=old_data.dtype,
        chunks=optimized_chunk_size,  # Optimized chunking
        track_times=False  # Reduces metadata overhead
    )

    new_data[:] = old_data[:]

    # Enable SWMR mode
    new_f.swmr_mode = True

print("New SWMR-enabled HDF5 file created.")


New SWMR-enabled HDF5 file created.


In [38]:
import h5py

file_path = "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1979.h5"

with h5py.File(file_path, "r", swmr=True) as f:
    swmr_enabled = f.swmr_mode
    libver = f.libver
    print(f"SWMR Enabled: {swmr_enabled}")
    print(f"File format version: {libver}")

SWMR Enabled: True
File format version: ('v110', 'v114')


In [22]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
os.environ["HDF5_CHUNK_CACHE_SIZE"] = str(100 * 1024 * 1024)  # 100MB

import time
import torch
from torch.utils.data import DataLoader, Dataset
import h5py

class HDF5Dataset(Dataset):
    def __init__(self, hdf5_file):
        self.hdf5_file = hdf5_file

    def __getitem__(self, index):
        with h5py.File(self.hdf5_file, "r") as f:
            return f["fields"][index]

    def __len__(self):
        with h5py.File(self.hdf5_file, "r") as f:
            return len(f["fields"])

dataset = HDF5Dataset("/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1979.h5")
dataloader = DataLoader(dataset, batch_size=32, num_workers=8, prefetch_factor=4, pin_memory=True)

start_time = time.time()
for batch in dataloader:
    pass  # Simulating data loading
end_time = time.time()

print(f"DataLoader Time (Before SWMR): {end_time - start_time:.2f} seconds")


DataLoader Time (Before SWMR): 33.88 seconds


In [21]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
os.environ["HDF5_CHUNK_CACHE_SIZE"] = str(100 * 1024 * 1024)  # 100MB
os.environ["HDF5_SWMR_WRITE"] = "1"
os.environ["HDF5_SWMR_READ"] = "1"

class HDF5SWMRDataset(Dataset):
    def __init__(self, hdf5_file):
        self.hdf5_file = hdf5_file
        self.f = h5py.File(self.hdf5_file, "r", libver="latest", swmr=True)

    def __getitem__(self, index):
        return self.f["fields"][index]

    def __len__(self):
        return len(self.f["fields"])

dataset_swmr = HDF5SWMRDataset("/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr/1979.h5")
dataloader_swmr = DataLoader(dataset_swmr, batch_size=32, num_workers=8, prefetch_factor=4, pin_memory=True)

start_time = time.time()
for batch in dataloader_swmr:
    pass
end_time = time.time()

print(f"DataLoader Time (After SWMR): {end_time - start_time:.2f} seconds")


DataLoader Time (After SWMR): 34.14 seconds


In [34]:
!lfs getstripe /iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train/1979.h5

/bin/bash: line 1: lfs: command not found


In [2]:
%run __common.py

Provided checkpoint directory /iopsstor/scratch/cscs/stefschu/DSM500/github/analysis/checkpoints does not exist, skipping load


Loaded validation datapipe of size 6


In [9]:
%load_ext autoreload
%autoreload 2

import torch
import gc
from modulus.datapipes.climate.era5_hdf5 import ERA5HDF5Datapipe
from torch.utils import data
import matplotlib.pyplot as plt

def list_cuda_tensors():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.is_cuda:
                print(type(obj), obj.size(), obj.dtype)
        except:
            pass  # Skip any errors

def print_memory_usage(tag=""):
    torch.cuda.synchronize()
    allocated = torch.cuda.memory_allocated() / 1024**2
    cached = torch.cuda.memory_reserved() / 1024**2
    print(f"[{tag}] Allocated: {allocated:.2f} MB | Cached: {cached:.2f} MB")

def tensor_memory(tensor):
    return tensor.element_size() * tensor.numel() / 1024**2  # Convert to MB

print_memory_usage("A")
datapipe = ERA5HDF5Datapipe(
    data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train",
    stats_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/stats",
    channels=list(range(21)),
    latlon_resolution=[721, 1440],
    interpolation_type=None,
    num_samples_per_year=1408,
    num_steps=10,
    num_history=0,
    use_cos_zenith=True,
    use_time_of_year_index=True,
    cos_zenith_args={
        "dt": 6.0,
        "start_year": 1979,
    },
    batch_size=1,
    num_workers=8,
    device=torch.device("cuda:0"),
    process_rank=0,
    world_size=1,
)

print_memory_usage("B")
#train_loader = data.DataLoader(train, batch_size=1, shuffle=True)

print_memory_usage("C")
import time
times = []
t0 = time.time()
for i, sample in enumerate(datapipe):
    times.append(time.time() - t0)
    t0 = time.time()
    #print(i, sample.shape, sample.dtype)
    #print(sample.mean(dim=(0, 1, 3, 4)), sample.std(dim=(0, 1, 3, 4)))
    #print()
    if i == 30:
        break

print_memory_usage("D")
print_memory_usage("E")

list_cuda_tensors()
print(tensor_memory(train.static_data))

import numpy as np
times = np.array(times)

print(np.mean(times), np.std(times))
# means = sample[0,:].mean(dim=(0, 2, 3))
# stds = sample[0,:].std(dim=(0, 2, 3))
# for m, s in zip(means, stds):
#     print(f"{m.item():.4f}, \t, {s.item():.4f}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[A] Allocated: 2702.00 MB | Cached: 2702.00 MB
[B] Allocated: 2702.00 MB | Cached: 2702.00 MB
[C] Allocated: 2702.00 MB | Cached: 2702.00 MB
[D] Allocated: 960.00 MB | Cached: 3662.00 MB
[E] Allocated: 960.00 MB | Cached: 3662.00 MB
<class 'torch.Tensor'> torch.Size([1, 21, 721, 1440]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 10, 21, 721, 1440]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 11, 1, 721, 1440]) torch.float32
217.8314208984375
0.8827576483449628 0.8889545674599225


/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1022: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)


In [42]:
# x = list(range(sample.shape[1]))
# y1 = sample[0, :, 27, 0, 0].cpu().to(torch.float32).numpy()
# y2 = sample[0, :, 28, 0, 0].cpu().to(torch.float32).numpy()

# x, y1, y2

# plt.figure(figsize=(10, 2))
# plt.plot(x, y1, label=f'sin(progress_in_year)')
# plt.plot(x, y2, label=f'cos(progress_in_year)')
# plt.xlabel("Time step in the year")
# plt.ylabel("Value")
# plt.title("Time of year")
# plt.legend()
# plt.grid(True)
# plt.show()

# x = list(range(9))
# y1 = sample[0, :9, 29, 0, 0].cpu().to(torch.float32).numpy()
# y2 = sample[0, :9, 30, 0, 0].cpu().to(torch.float32).numpy()

# x, y1, y2

# plt.figure(figsize=(10, 2))
# plt.plot(x, y1, label=f'sin(progress_in_day)')
# plt.plot(x, y2, label=f'cos(progress_in_day)')
# plt.xlabel("Time step in the year")
# plt.ylabel("Value")
# plt.title("Time of year")
# plt.legend()
# plt.grid(True)
# plt.show()

IndexError: index 1 is out of bounds for dimension 0 with size 1

<Figure size 1000x200 with 0 Axes>

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from modulus.datapipes.climate.era5_hdf5_new_dali import ERA5HDF5Datapipe

datapipe = ERA5HDF5Datapipe(
    data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr",
    statistics_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/stats",
    static_data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/static",
    num_steps_ahead=10,
    device="cuda:0",
    dtype=torch.bfloat16
)

Time to init ERA5HDF5Datapipe: 0.00s
ERA5HDF5Datapipe.__init__ 218650 709.5


In [ ]:
import time
t0 = time.time()
for sample in datapipe:
    print(len(sample), sample[0]["input"].shape, sample[0]["output"].shape, f"{(time.time() - t0):.2f} s")
    t0 = time.time()
    #print(sample.shape)

NameError: name 'datapipe' is not defined

In [62]:
%load_ext autoreload
%autoreload 2

# distributed manager initialization (by mocking a simple slurm job)
from modulus.distributed import DistributedManager
import os
import random
os.environ["MODULUS_DISTRIBUTED_INITIALIZATION_METHOD"] = "ENV"
os.environ["MASTER_PORT"] = str(random.randint(10000, 20000)) # <--- Several notebooks can be running at the same time, none of which is running anything distributed
os.environ["MASTER_ADDR"] = "localhost"
os.environ["WORLD_SIZE"] = "1"
os.environ["RANK"] = "0"

DistributedManager.initialize()
dist = DistributedManager()

import torch
from modulus.datapipes.climate.era5_hdf5_new_dali_mem import ERA5HDF5Datapipe

datapipe = ERA5HDF5Datapipe(
    data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/train_swmr",
    statistics_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/stats",
    static_data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/static",
    dist=dist,
)

print("XXX", flush=True)

import time
from datetime import datetime

t0 = time.time()
for i, sample in enumerate(datapipe):
    print(g(sample[0]["input_timestamps"]), sample[0]["input"].shape, sample[0]["output"].shape, f"{(time.time() - t0):.2f} s")
    t0 = time.time()
    if i == 2:
        break

sample = sample[0]
xsample = torch.concatenate([sample["input"], sample["output"]], dim=1)[0]
xsample.shape

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/iopsstor/scratch/cscs/stefschu/DSM500/github/modulus-a5275d8/modulus/distributed/manager.py:329: UserWarning: Distributed manager is already intialized
  warn("Distributed manager is already intialized")


XXX
tensor([[1.3172e+09]], dtype=torch.float64) 1317160800
2011-09-28 00:00:00 torch.Size([1, 1, 31, 721, 1440]) torch.Size([1, 1, 31, 721, 1440]) 4.23 s
tensor([[8.5639e+08]], dtype=torch.float64) 856393200
1997-02-20 00:00:00 torch.Size([1, 1, 31, 721, 1440]) torch.Size([1, 1, 31, 721, 1440]) 0.14 s
tensor([[7.0092e+08]], dtype=torch.float64) 700916400
1992-03-18 12:00:00 torch.Size([1, 1, 31, 721, 1440]) torch.Size([1, 1, 31, 721, 1440]) 0.11 s


torch.Size([2, 31, 721, 1440])

In [64]:
sample["input_timestamps"].item(), sample["output_timestamps"].item()

(700916400.0, 700938000.0)

In [52]:
import numpy as np

arr = np.array([1, 2, 3, 4, 5])
np.random.shuffle(arr)

print(arr)  # The order is changed in place

[3 2 4 5 1]


In [68]:
import sys
sys.path.append('/iopsstor/scratch/cscs/stefschu/DSM500/github/modulus-a5275d8')

['/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/iopsstor/scratch/cscs/stefschu/DSM500/github/venv_arm64/lib/python3.10/site-packages',
 '__editable__.nvidia_modulus-0.10.0a0.finder.__path_hook__',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/nvfuser-0.2.10a0+f669fcf-py3.10-linux-aarch64.egg',
 '/usr/local/lib/python3.10/dist-packages/lightning_thunder-0.2.0.dev0-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/dill-0.3.9-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/opt_einsum-3.4.0-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/igraph-0.11.6-py3.10-linux-aarch64.egg',
 '/usr/local/lib/python3.10/dist-packages/lightning_utilities-0.11.7-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/looseversion-1.3.0-py3.10.egg',
 '/usr/local/lib/python3.10/dist-packages/texttable-1.7.0-py3.10.egg',
 '/usr/lib/python3/dist-packages',
 '/iopsstor/scratch/cscs/stefschu/DSM500/github/ven

In [70]:
output = np.ones((1, 3, 31, 721, 1440))
prediction = np.zeros((1, 21, 721, 1440))

output.shape, prediction.shape

((1, 3, 31, 721, 1440), (1, 21, 721, 1440))

In [76]:
temp = prediction.reshape(1, 1, 21, 721, 1440)
print(temp.shape)

np.all(temp[0] == prediction)

(1, 1, 21, 721, 1440)


True

In [79]:
output[:, [1], 21:].shape

(1, 1, 10, 721, 1440)

In [89]:
a = np.concatenate([temp, output[:, [1], 21:]], axis=2)
print(a.shape)
for i in enumerate(a.sum(axis=(0, 1, 3, 4))):
    print(i)

(1, 1, 31, 721, 1440)
(0, 0.0)
(1, 0.0)
(2, 0.0)
(3, 0.0)
(4, 0.0)
(5, 0.0)
(6, 0.0)
(7, 0.0)
(8, 0.0)
(9, 0.0)
(10, 0.0)
(11, 0.0)
(12, 0.0)
(13, 0.0)
(14, 0.0)
(15, 0.0)
(16, 0.0)
(17, 0.0)
(18, 0.0)
(19, 0.0)
(20, 0.0)
(21, 1038240.0)
(22, 1038240.0)
(23, 1038240.0)
(24, 1038240.0)
(25, 1038240.0)
(26, 1038240.0)
(27, 1038240.0)
(28, 1038240.0)
(29, 1038240.0)
(30, 1038240.0)


# Study why validation takes twice as long with the new dataloader

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import hydra
with hydra.initialize(config_path=".", version_base="1.3"):
    cfg = hydra.compose(config_name="config")

import os
import random
from modulus.distributed import DistributedManager
os.environ["MODULUS_DISTRIBUTED_INITIALIZATION_METHOD"] = "ENV"
os.environ["MASTER_PORT"] = str(random.randint(10000, 20000)) # <--- Several notebooks can be running at the same time, none of which is running anything distributed
os.environ["MASTER_ADDR"] = "localhost"
os.environ["WORLD_SIZE"] = "1"
os.environ["RANK"] = "0"

DistributedManager.initialize()
dist = DistributedManager()

In [3]:
import sys
sys.path.append('/iopsstor/scratch/cscs/stefschu/DSM500/github/modulus-a5275d8/examples/weather/graphcast')

from modulus.launch.logging import (
    PythonLogger,
    RankZeroLoggingWrapper
)
from omegaconf import DictConfig
logger = PythonLogger("main")  # General python logger
rank_zero_logger = RankZeroLoggingWrapper(logger, dist)  # Rank 0 logger
rank_zero_logger.file_logging()

# Instantiate dummy trainer
from train_graphcast import GraphCastTrainer
trainer = GraphCastTrainer(cfg, dist, rank_zero_logger)

Provided checkpoint directory /iopsstor/scratch/cscs/stefschu/DSM500/github/analysis/checkpoints does not exist, skipping load


FusedAdam not available, using AdamW
Loaded validation datapipe of size 2912


## New

In [27]:
from modulus.datapipes.climate.era5_hdf5_new import ERA5HDF5Datapipe

val_datapipe = ERA5HDF5Datapipe(
    data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/test_swmr",
    statistics_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/stats",
    static_data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/static",
    num_steps=cfg.num_val_steps,
    dist=dist,
)

import time

t0 = time.time()
mse = 0
torch.cuda.empty_cache()
print(f"Pipeline length: {len(val_datapipe)}")
for i, data in enumerate(val_datapipe):
    print(data[0]["input"].shape, data[0]["input"].dtype, data[0]["output"].shape, data[0]["output"].dtype)
    
    if i > 10: # cfg.num_val_spy:
        break

    # Prepare the input & output
    input = data[0]["input"].to(dtype=torch.bfloat16)
    output = data[0]["output"]

    predictions = (
        torch.empty(output.shape)
        .to(dtype=torch.bfloat16)
        .to(device=dist.device)
    )
    print(f"{i}: Data loading + prep: {time.time() - t0:.2f} s, going to loop {output.size(dim=1)} times")
    #t0 = time.time()
    for t in range(output.size(dim=1)):
        #_pred = input[0, :, :21]
        #_pred = trainer.model(input)

        #_pred = _pred.reshape(1, 1, 21, 721, 1440)
        #_pred = torch.cat([_pred, output[:, [t], 21:]], axis=2)

        #predictions[0, t] = _pred
        #input = _pred
        time.sleep(.5)
    t0 = time.time()
    mse += torch.mean(torch.pow(predictions - output, 2))
    torch.cuda.nvtx.range_pop()

    #del input, _pred
    torch.cuda.empty_cache()

del predictions, output

mse


Pipeline length: 2912
torch.Size([1, 1, 31, 721, 1440]) torch.float32 torch.Size([1, 8, 31, 721, 1440]) torch.float32
0: Data loading + prep: 4.47 s, going to loop 8 times
torch.Size([1, 1, 31, 721, 1440]) torch.float32 torch.Size([1, 8, 31, 721, 1440]) torch.float32
1: Data loading + prep: 0.06 s, going to loop 8 times
torch.Size([1, 1, 31, 721, 1440]) torch.float32 torch.Size([1, 8, 31, 721, 1440]) torch.float32
2: Data loading + prep: 0.18 s, going to loop 8 times
torch.Size([1, 1, 31, 721, 1440]) torch.float32 torch.Size([1, 8, 31, 721, 1440]) torch.float32
3: Data loading + prep: 0.27 s, going to loop 8 times
torch.Size([1, 1, 31, 721, 1440]) torch.float32 torch.Size([1, 8, 31, 721, 1440]) torch.float32
4: Data loading + prep: 0.25 s, going to loop 8 times
torch.Size([1, 1, 31, 721, 1440]) torch.float32 torch.Size([1, 8, 31, 721, 1440]) torch.float32
5: Data loading + prep: 0.18 s, going to loop 8 times
torch.Size([1, 1, 31, 721, 1440]) torch.float32 torch.Size([1, 8, 31, 721, 144

tensor(10.1059, device='cuda:0')

## Old

In [29]:
from modulus.datapipes.climate import ERA5HDF5Datapipe
from train_utils import prepare_input
from modulus.utils.graphcast.data_utils import StaticData

latitudes = trainer.model.latitudes
longitudes = trainer.model.longitudes
static_data = StaticData(
    "/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/static", latitudes, longitudes
).get()
static_data = static_data.to(device=dist.device)

interpolation_type = None
cos_zenith_args = {
    "dt": 6.0,
    "start_year": 2017,
}
val_datapipe = ERA5HDF5Datapipe(
    data_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/test_swmr",
    stats_dir="/iopsstor/scratch/cscs/stefschu/DSM500/data/FCN_ERA5_data_v0/stats",
    channels=[i for i in range(cfg.num_channels_climate)],
    latlon_resolution=cfg.latlon_res,
    interpolation_type=interpolation_type,
    num_steps=cfg.num_val_steps,
    num_history=cfg.num_history,
    use_cos_zenith=cfg.use_cos_zenith,
    use_time_of_year_index=cfg.use_time_of_year_index,
    cos_zenith_args=cos_zenith_args,
    batch_size=1,
    num_samples_per_year=5,#cfg.num_val_spy,
    shuffle=False,
    device=dist.device,
    process_rank=dist.rank,
    world_size=dist.world_size,
    num_workers=cfg.num_workers,
)

num_history = cfg.num_history
stride = cfg.stride
dt = cfg.dt
num_samples_per_year_train = cfg.num_samples_per_year_train

import time

loss_epoch = 0
prepare_input_vars = {
    "num_history": num_history,
    "static_data": static_data,
    "stride": stride,
    "dt": dt,
    "num_samples_per_year": num_samples_per_year_train,
    "device": dist.device,
}
t0 = time.time()
torch.cuda.empty_cache()
print(f"Pipeline length: {len(val_datapipe)}")
for i, data in enumerate(val_datapipe):
    print(data[0]["invar"].shape, data[0]["invar"].dtype, data[0]["outvar"].shape, data[0]["outvar"].dtype)
    invar = data[0]["invar"]
    outvar = data[0]["outvar"][0]
    try:
        cos_zenith = data[0]["cos_zenith"]
    except KeyError:
        cos_zenith = None
    try:
        time_idx = data[0]["time_of_year_idx"].item()
    except KeyError:
        time_idx = None
    invar_cat = prepare_input(
        invar=invar,
        cos_zenith=cos_zenith,
        time_idx=time_idx,
        **prepare_input_vars,
        step=1,
    )
    torch.cuda.empty_cache()
    invar_cat = invar_cat.to(dtype=torch.bfloat16)

    pred = (
        torch.empty(outvar.shape)
        .to(dtype=torch.bfloat16)
        .to(device=dist.device)
    )
    print(f"{i}: Data loading + prep: {time.time() - t0:.2f} s, going to loop {outvar.shape[0]} times")
    
    for t in range(outvar.shape[0]):
        time.sleep(.5)
    t0 = time.time()
    loss_epoch += torch.mean(torch.pow(pred - outvar, 2))
    torch.cuda.nvtx.range_pop()

    #del invar, outpred
    torch.cuda.empty_cache()

Pipeline length: 10
Time to extract data: 0.24067306518554688
Time to extract data: 0.2447528839111328
Time to extract data: 0.2325267791748047
torch.Size([1, 21, 721, 1440]) torch.float32 torch.Size([1, 8, 21, 721, 1440]) torch.float32
0: Data loading + prep: 4.16 s, going to loop 8 times
Time to extract data: 0.24914026260375977
torch.Size([1, 21, 721, 1440]) torch.float32 torch.Size([1, 8, 21, 721, 1440]) torch.float32
1: Data loading + prep: 0.21 s, going to loop 8 times
Time to extract data: 0.2470099925994873
torch.Size([1, 21, 721, 1440]) torch.float32 torch.Size([1, 8, 21, 721, 1440]) torch.float32
2: Data loading + prep: 0.26 s, going to loop 8 times
Time to extract data: 0.46944570541381836
torch.Size([1, 21, 721, 1440]) torch.float32 torch.Size([1, 8, 21, 721, 1440]) torch.float32
3: Data loading + prep: 0.24 s, going to loop 8 times
Time to extract data: 0.23991680145263672
torch.Size([1, 21, 721, 1440]) torch.float32 torch.Size([1, 8, 21, 721, 1440]) torch.float32
4: Data 

Process SpawnProcess-196:
Process SpawnProcess-199:
Process SpawnProcess-195:
Process SpawnProcess-194:
Process SpawnProcess-197:
Process SpawnProcess-193:
Process SpawnProcess-200:
Process SpawnProcess-198:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 462, in worker
    scheduled, shm_chunk = worker_context.get_task()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 434, in get_task
    scheduled_meta = self.task_receiver.get_task()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/worker.py", line 132, in get_task
    recv = self.queue.get()
  File "/usr/local/lib/python3.10/dist-packages/nvidia/dali/_multiproc/shared_queue.py", line 17

## Memory check

In [12]:
import gc

def list_cuda_tensors():
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.is_cuda:
                print(type(obj), obj.size(), obj.dtype)
        except:
            pass  # Skip any errors

def print_memory_usage(tag=""):
    torch.cuda.synchronize()
    allocated = torch.cuda.memory_allocated() / 1024**2
    cached = torch.cuda.memory_reserved() / 1024**2
    print(f"[{tag}] Allocated: {allocated:.2f} MB | Cached: {cached:.2f} MB")

list_cuda_tensors()
print_memory_usage()

<class 'torch.Tensor'> torch.Size([1, 1, 31, 721, 1440]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 8, 31, 721, 1440]) torch.float32
<class 'torch.Tensor'> torch.Size([1, 8]) torch.float64
<class 'torch.Tensor'> torch.Size([1, 1]) torch.float64
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 512]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 512]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 512]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 4]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 512]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512]) torch.bfloat16
<class 'torch.nn.parameter.Parameter'> torch.Size([512]) torch.bfloat16
<cl

In [1]:
import numpy as np

a = np.array(range(1000, 1016), dtype=np.float32)
a.astype(np.int32).tolist()

[1000,
 1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015]

(19.53125, dtype('uint16'), 9999)

In [19]:
indices = np.arange(10000, dtype=np.uint16)
indices.nbytes / 1024, indices.dtype, indices.max()
np.random.default_rng(seed=4).shuffle(indices)
indices = np.array_split(indices, 4)
indices = indices[2]
indices.nbytes / 1024, indices.dtype, indices.max()

(4.8828125, dtype('uint16'), 9999)

In [ ]:
self.indices = np.arange(self.num_samples, dtype=np.uint16)
        if shuffled:
            seed = sample_info.epoch_idx + int(os.getenv("SLURM_JOB_ID", 0))
            np.random.default_rng(seed=seed).shuffle(self.indices)
        self.indices = np.array_split(self.indices, self.dist.world_size)
        self.indices = self.indices[self.dist.rank]